In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book() # to connect the colab notebook to google drive using an authentication token
from fastai.vision.all import *

     |████████████████████████████████| 720 kB 13.9 MB/s 
     |████████████████████████████████| 189 kB 62.9 MB/s 
     |████████████████████████████████| 1.2 MB 65.4 MB/s 
     |████████████████████████████████| 46 kB 5.0 MB/s 
     |████████████████████████████████| 56 kB 5.4 MB/s 
     |████████████████████████████████| 51 kB 416 kB/s 
Mounted at /content/gdrive


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/fastai-practic
!git add .
!git commit -m "setting chap2"
!git push

[Errno 107] Transport endpoint is not connected: '/content/drive/MyDrive/Colab Notebooks/fastai-practic'
/content
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
